## The whole script

In [5]:
""" #TODO

"""

import glob
import multiprocessing
import os
import sys
import time
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from daplis.functions import calc_diff as cd
from daplis.functions import utils
from daplis.functions.calibrate import load_calibration_data
from numpy import ndarray
from pyarrow import feather as ft


class MpWizard:

    # Initialize by passing the input parameters which later will be
    # passed into all internal functions
    def __init__(
        self,
        path: str = "",
        pixels: list = [],
        daughterboard_number: str = "",
        motherboard_number: str = "",
        firmware_version: str = "",
        timestamps: int = 512,
        delta_window: float = 50e3,
        include_offset: bool = False,
        apply_calibration: bool = True,
        apply_mask: bool = True,
        absolute_timestamps: bool = False,
        number_of_cores: int = 1,
    ):
        """Initialization of the class.

        Set all the input parameters for later use with internal class
        functions. Additionally, preload the calibration matrix.

        Parameters
        ----------
        path : str, optional
            _description_, by default ""
        pixels : list, optional
            _description_, by default []
        daughterboard_number : str, optional
            _description_, by default ""
        motherboard_number : str, optional
            _description_, by default ""
        firmware_version : str, optional
            _description_, by default ""
        timestamps : int, optional
            _description_, by default 512
        delta_window : float, optional
            _description_, by default 50e3
        include_offset : bool, optional
            _description_, by default False
        apply_calibration : bool, optional
            _description_, by default True
        apply_mask : bool, optional
            _description_, by default True
        absolute_timestamps : bool, optional
            _description_, by default False
        number_of_cores : int, optional
            _description_, by default 1
        """

        self.path = path
        self.pixels = pixels
        self.daughterboard_number = daughterboard_number
        self.motherboard_number = motherboard_number
        self.firmware_version = firmware_version
        self.timestamps = timestamps
        self.delta_window = delta_window
        self.include_offset = include_offset
        self.apply_calibration = apply_calibration
        self.apply_mask = apply_mask
        self.absolute_timestamps = absolute_timestamps
        self.number_of_cores = number_of_cores

        os.chdir(self.path)

        # Load calibration if requested
        if self.apply_calibration:

            # work_dir = Path(__file__).resolve().parent.parent

            # TODO
            # path_calibration_data = os.path.join(
            #     work_dir, r"params\calibration_data"
            # )
            path_calibration_data = (
                r"/home/sj/GitHub/daplis/src/daplis/params/calibration_data"
            )
            # path_calibration_data = r"C:\Users\bruce\Documents\GitHub\daplis\src\daplis\params\calibration_data"

            calibration_data = load_calibration_data(
                path_calibration_data,
                daughterboard_number,
                motherboard_number,
                firmware_version,
                include_offset,
            )

            if self.include_offset:
                self.calibration_matrix, self.offset_array = calibration_data
            else:
                self.calibration_matrix = calibration_data

        # Apply mask if requested
        if self.apply_mask:
            mask = utils.apply_mask(
                self.daughterboard_number,
                self.motherboard_number,
            )
            if isinstance(self.pixels[0], int) and isinstance(
                self.pixels[1], int
            ):
                self.pixels = [pix for pix in self.pixels if pix not in mask]
            else:
                self.pixels = [
                    [value for value in sublist if value not in mask]
                    for sublist in pixels
                ]

        # Check the firmware version and set the pixel coordinates accordingly
        if self.firmware_version == "2212s":
            self.pix_coor = np.arange(256).reshape(4, 64).T
        elif firmware_version == "2212b":
            self.pix_coor = np.arange(256).reshape(64, 4)
        else:
            print("\nFirmware version is not recognized.")
            sys.exit()

    def _unpack_binary_data(
        self,
        file: str,
    ) -> np.ndarray:
        """Unpack binary data from LinoSPAD2.

        Same unpacking function as the standard 'daplis' one, except
        the calibration matrix is preloaded during initialization and
        called here. Return a 3D matrix of pixel numbers and timestamps.

        Parameters
        ----------
        file : str
            A '.dat' data file from LinoSPAD2 with the binary-encoded
            data.

        Returns
        -------
        np.ndarray
            A 3D matrix of the pixel numbers where timestamp was
            recorded and timestamps themselves.
        """
        # Unpack binary data
        raw_data = np.memmap(file, dtype=np.uint32)
        # Timestamps are stored in the lower 28 bits
        data_timestamps = (raw_data & 0xFFFFFFF).astype(np.int64)
        # Pixel address in the given TDC is 2 bits above timestamp
        data_pixels = ((raw_data >> 28) & 0x3).astype(np.int8)
        # Check the top bit, assign '-1' to invalid timestamps
        data_timestamps[raw_data < 0x80000000] = -1

        # Number of acquisition cycles in each data file
        cycles = len(data_timestamps) // (self.timestamps * 65)
        # Transform into a matrix of size 65 by cycles*timestamps
        data_pixels = (
            data_pixels.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        data_timestamps = (
            data_timestamps.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        # Cut the 65th TDC that does not hold any actual data from pixels
        data_pixels = data_pixels[:-1]
        data_timestamps = data_timestamps[:-1]

        # Insert '-2' at the end of each cycle
        insert_indices = np.linspace(
            self.timestamps, cycles * self.timestamps, cycles
        ).astype(np.int64)

        data_pixels = np.insert(
            data_pixels,
            insert_indices,
            -2,
            1,
        )
        data_timestamps = np.insert(
            data_timestamps,
            insert_indices,
            -2,
            1,
        )

        # Combine both matrices into a single one, where each cell holds pixel
        # coordinates in the TDC and the timestamp
        data_all = np.stack((data_pixels, data_timestamps), axis=2).astype(
            np.int64
        )

        if self.apply_calibration is False:
            data_all[:, :, 1] = data_all[:, :, 1] * 2500 / 140
        else:
            # Path to the calibration data
            pix_coordinates = np.arange(256).reshape(64, 4)
            for i in range(256):
                # Transform pixel number to TDC number and pixel
                # coordinates in that TDC (from 0 to 3)
                tdc, pix = np.argwhere(pix_coordinates == i)[0]
                # Find data from that pixel
                ind = np.where(data_all[tdc].T[0] == pix)[0]
                # Cut non-valid timestamps ('-1's)
                ind = ind[data_all[tdc].T[1][ind] >= 0]
                if not np.any(ind):
                    continue
                data_cut = data_all[tdc].T[1][ind]
                # Apply calibration; offset is added due to how delta
                # ts are calculated
                if self.include_offset:
                    data_all[tdc].T[1][ind] = (
                        (data_cut - data_cut % 140) * 2500 / 140
                        + self.calibration_matrix[i, (data_cut % 140)]
                        + self.offset_array[i]
                    )
                else:
                    data_all[tdc].T[1][ind] = (
                        data_cut - data_cut % 140
                    ) * 2500 / 140 + self.calibration_matrix[
                        i, (data_cut % 140)
                    ]

        return data_all

    def _calculate_differences_2212_fast(
        self,
        data: ndarray,
        delta_window: float = 50e3,
        cycle_length: float = 4e9,
    ):
        """Calculate timestamp differences for firmware version 2212.

        Calculate timestamp differences for the given pixels and LinoSPAD2
        firmware version 2212. Modified compared to the standard 'daplis'
        one. Modifications are for working with prechunked data, i.e.,
        sliced down to two arrays from the whole 64xN matrix.

        Parameters
        ----------
        data : ndarray
            Matrix of timestamps, where rows correspond to the TDCs.
        pixels : List[int] | List[List[int]]
            List of pixel numbers for which the timestamp differences should
            be calculated or list of two lists with pixel numbers for peak
            vs. peak calculations.
        pix_coor : ndarray
            Array for transforming the pixel address in terms of TDC (0 to 3)
            to pixel number in terms of half of the sensor (0 to 255).
        delta_window : float, optional
            Width of the time window for counting timestamp differences.
            The default is 50e3 (50 ns).
        cycle_length : float, optional
            Length of each acquisition cycle. The default is 4e9 (4 ms).

        Returns
        -------
        deltas_all : dict
            Dictionary containing timestamp differences for each pair of pixels.

        """

        # Dictionary for the timestamp differences, where keys are the
        # pixel numbers of the requested pairs

        deltas_all = []

        data_pix_1 = data[0]
        data_pix_2 = data[1]

        indices1 = data_pix_1[0]
        indices2 = data_pix_2[0]
        timestamps1 = data_pix_1[1]
        timestamps2 = data_pix_2[1]

        timestamps_1 = []
        timestamps_2 = []

        # Go over cycles, shifting the timestamps from each next
        # cycle by lengths of cycles before (e.g., for the 4th cycle
        # add 12 ms)
        for i, _ in enumerate(self.cycle_ends[:-1]):
            slice_from = self.cycle_ends[i]
            slice_to = self.cycle_ends[i + 1]
            pix1_slice = indices1[
                (indices1 >= slice_from) & (indices1 < slice_to)
            ]
            if not np.any(pix1_slice):
                continue
            pix2_slice = indices2[
                (indices2 >= slice_from) & (indices2 < slice_to)
            ]
            if not np.any(pix2_slice):
                continue

            # Shift timestamps by cycle length
            tmsp1 = timestamps1[np.isin(indices1, pix1_slice)]
            tmsp1 = tmsp1[tmsp1 > 0]
            tmsp1 = tmsp1 + cycle_length * i

            tmsp2 = timestamps2[np.isin(indices2, pix2_slice)]
            tmsp2 = tmsp2[tmsp2 > 0]
            tmsp2 = tmsp2 + cycle_length * i

            timestamps_1.extend(tmsp1)
            timestamps_2.extend(tmsp2)

        timestamps_1 = np.array(timestamps_1)
        timestamps_2 = np.array(timestamps_2)

        # Indicators for each pixel: 0 for timestamps from one pixel
        # 1 - from the other
        pix1_ind = np.zeros(len(timestamps_1), dtype=np.int32)
        pix2_ind = np.ones(len(timestamps_2), dtype=np.int32)

        pix1_data = np.vstack((pix1_ind, timestamps_1))
        pix2_data = np.vstack((pix2_ind, timestamps_2))

        # Dataframe for each pixel with pixel indicator and
        # timestamps
        df1 = pd.DataFrame(pix1_data.T, columns=["Pixel_index", "Timestamp"])
        df2 = pd.DataFrame(pix2_data.T, columns=["Pixel_index", "Timestamp"])

        # Combine the two dataframes
        df_combined = pd.concat((df1, df2), ignore_index=True)

        # Sort the timestamps
        df_combined.sort_values("Timestamp", inplace=True)

        # Subtract pixel indicators of neighbors; values of 0
        # correspond to timestamp differences for the same pixel
        # '-1' and '1' - to differences from different pixels
        df_combined["Pixel_index_diff"] = df_combined["Pixel_index"].diff()

        # Calculate timestamp difference between neighbors
        df_combined["Timestamp_diff"] = df_combined["Timestamp"].diff()

        # Get the correct timestamp difference sign
        df_combined["Timestamp_diff"] = (
            df_combined["Timestamp_diff"] * df_combined["Pixel_index_diff"]
        )

        # Collect timestamp differences where timestamps are from
        # different pixels
        filtered_df = df_combined[abs(df_combined["Pixel_index_diff"]) == 1]

        # Save only timestamps differences in the requested window
        delta_ts = filtered_df[
            abs(filtered_df["Timestamp_diff"]) < delta_window
        ]["Timestamp_diff"].values

        deltas_all.extend(delta_ts)

        return deltas_all

    def _calculate_timestamps_differences(self, args):
        """Calculate photon coincidences and save to '.feather'.

        Parameters
        ----------
        #TODO
        """

        print("From The Process", time.time())

        # try-except railguard for a function that goes to separate
        # cores
        file, data, pixel_pair = args
        try:
            # Check if the 'delta_ts_data' folder exists
            output_dir = Path(self.path) / "delta_ts_data"
            output_dir.mkdir(exist_ok=True)

            # Calculate the differences and convert them to a pandas
            # dataframe - standard approach sending the whole matrix of
            # 64xN to each child process
            # deltas_all = cd.calculate_differences_2212_fast(
            #     data, pixel_pair, self.pix_coor
            # )

            # Calculate the differences and convert them to a pandas
            # dataframe - pre-chunking the data down to 2 arrays
            deltas_all = self._calculate_differences_2212_fast(data)
            data_for_plot_df = pd.DataFrame(
                deltas_all, columns=[f"{pixel_pair[0]},{pixel_pair[1]}"]
            ).T

            # Save the data to a '.feather' file
            file_name = Path(
                file
            ).stem  # Get the file name without the extension
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            file_name = Path(file).stem
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            ft.write_feather(
                data_for_plot_df.reset_index(drop=True), output_file
            )

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    def _combine_feather_files(self, path_to_feather_files: str):

        os.chdir(path_to_feather_files)

        for pixel_pair in self.pixel_pairs:
            ft_files = glob.glob(f"*{pixel_pair[0]}_{pixel_pair[1]}*.feather")
            data_all = pd.DataFrame()
            for ft_file in ft_files:
                data = ft.read_feather(ft_file)
                data_all = pd.concat((data_all, data), ignore_index=True)
            data_all.to_feather(
                f"combined_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )

    ### Heavy data chunking - fast calculation in each child process at
    ### the cost of long chunking - no acceleration as a result
    # def _chunk_that_data_too_much(self, data, pixel_pair, cycle_length: float = 4e9):

    #     pix_coor = np.arange(256).reshape(64, 4)

    #     # Find ends of cycles
    #     cycle_ends = np.argwhere(data[0].T[0] == -2)
    #     cycle_ends = np.insert(cycle_ends, 0, 0)

    #     tdc1, pix_c1 = np.argwhere(pix_coor == pixel_pair[0])[0]
    #     pix1 = np.where(data[tdc1].T[0] == pix_c1)[0]

    #     timestamps_1 = []
    #     timestamps_2 = []

    #     # Second pixel in the pair
    #     tdc2, pix_c2 = np.argwhere(pix_coor == pixel_pair[1])[0]
    #     pix2 = np.where(data[tdc2].T[0] == pix_c2)[0]

    #     # Go over cycles, shifting the timestamps from each next
    #     # cycle by lengths of cycles before (e.g., for the 4th cycle
    #     # add 12 ms)
    #     for i, _ in enumerate(cycle_ends[:-1]):
    #         slice_from = cycle_ends[i]
    #         slice_to = cycle_ends[i + 1]
    #         pix1_slice = pix1[(pix1 >= slice_from) & (pix1 < slice_to)]
    #         if not np.any(pix1_slice):
    #             continue
    #         pix2_slice = pix2[(pix2 >= slice_from) & (pix2 < slice_to)]
    #         if not np.any(pix2_slice):
    #             continue

    #         # Shift timestamps by cycle length
    #         tmsp1 = data[tdc1].T[1][pix1_slice]
    #         tmsp1 = tmsp1[tmsp1 > 0]
    #         tmsp1 = tmsp1 + cycle_length * i

    #         tmsp2 = data[tdc2].T[1][pix2_slice]
    #         tmsp2 = tmsp2[tmsp2 > 0]
    #         tmsp2 = tmsp2 + cycle_length * i

    #         timestamps_1.extend(tmsp1)
    #         timestamps_2.extend(tmsp2)

    #     timestamps_1 = np.array(timestamps_1)
    #     timestamps_2 = np.array(timestamps_2)

    #     return (timestamps_1, timestamps_2)

    def _chunk_that_data(self, data, pixel_pair, cycle_length: float = 4e9):
        """Chunk data down to 2 rows - prepare for child processes.

        Parameters
        ----------
        data : np.ndarray
            The whole matrix.
        pixel_pair : list
            Pair of pixels to slice out of the whole matrix.
        cycle_length : float, optional
            Cycle length in ps, by default 4e9

        Returns
        -------
        tuple
            Tuple of two arrays, each contains timestamps from the two
            pixels requested and the indices of the timestamps in the
            original matrix of data. The indices are used for correct
            assigning to the corresponding acquisition cycles.
        """

        tdc1, pix_c1 = np.argwhere(self.pix_coor == pixel_pair[0])[0]
        indices1 = np.where(data[tdc1].T[0] == pix_c1)[0]

        # Second pixel in the pair
        tdc2, pix_c2 = np.argwhere(self.pix_coor == pixel_pair[1])[0]
        indices2 = np.where(data[tdc2].T[0] == pix_c2)[0]

        data_cut_1 = np.array((indices1, data[tdc1].T[1][indices1]))
        data_cut_2 = np.array((indices2, data[tdc2].T[1][indices2]))

        return (data_cut_1, data_cut_2)

    def calculate_and_save_timestamp_differences_mp(self):

        # Find all LinoSPAD2 data files
        files = glob.glob("*.dat")

        if not files:
            raise ValueError("No .dat files found in the specified path.")

        self.pixel_pairs = []
        for i in self.pixels[0]:
            for j in self.pixels[1]:
                self.pixel_pairs.append([i, j])

        start_time = time.time()

        # Go file by file
        for file in files:
            print(f"Processing file: {file}")

            start_time_unpack = time.time()

            # Unpack the data from the file
            data = self._unpack_binary_data(file)

            # Pre-collect the indices of the acquisition cycles' ends
            self.cycle_ends = np.argwhere(data[0].T[0] == -2)
            self.cycle_ends = np.insert(self.cycle_ends, 0, 0)

            start_time_args = time.time()

            # Prepare the arguments for the child processes:
            # 'file' and 'pixel_pair' for naming purposes,
            # chunked data as a tuple of two arrays to work with
            args = [
                (
                    file,
                    self._chunk_that_data(data, pixel_pair, 250e6),
                    pixel_pair,
                )
                for pixel_pair in self.pixel_pairs
            ]

            # print(f"Created args in {time.time() - start_time_args}")
            print("Before the processes", time.time())

            with multiprocessing.Pool(
                min(self.number_of_cores, len(self.pixel_pairs))
            ) as pool:
                pool.map(
                    self._calculate_timestamps_differences,
                    args,
                )

        end_time = time.time()

        print(
            f"Parallel processing of files "
            "files (with each writing to its file) finished "
            f"in: {round(end_time - start_time, 2)} s"
        )

        # Combine '.feather' files from separate cores
        path_to_feathers = os.path.join(self.path, "delta_ts_data")

        self._combine_feather_files(path_to_feathers)

        path_output = os.path.join(self.path, "delta_ts_data")

        print(
            "The feather files with the timestamp differences were "
            f"combined into the 'combined.feather' file in {path_output}"
        )


if __name__ == "__main__":
    # path = r"D:\LinoSPAD2\Data\B7d\2024.11.11\500t\MP_test"
    path = r'/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70'

    mp = MpWizard(
        path,
        # pixels=[[144], [171, 172]],
        pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
        # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
        daughterboard_number="B7d",
        motherboard_number="#28",
        firmware_version="2212b",
        timestamps=500,
        number_of_cores=5,
    )

    mp.calculate_and_save_timestamp_differences_mp()


### Standard approach - for control and comparison

# import time

# from daplis.functions import delta_t

# time_start = time.time()

# path = r"/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70"


# delta_t.calculate_and_save_timestamp_differences_fast(
#     path,
#     rewrite=True,
#     # pixels=[144, 171],
#     pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
#     # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
#     daughterboard_number="B7d",
#     motherboard_number="#28",
#     firmware_version="2212b",
#     timestamps=500,
# )

# print(f"Finished in {time.time() - time_start}")

Processing file: 0000054222.dat
Before the processes 1737458797.9101393
From The Process 1737458797.9579823
From The Process 1737458797.9676855
From The Process 1737458797.9822028
From The Process 1737458798.0107205
From The Process 1737458798.0298026
From The Process 1737458798.5469003
From The Process 1737458798.5578787
From The Process 1737458798.581703
From The Process 1737458798.7217958
From The Process 1737458799.0385582
From The Process 1737458799.1499593
From The Process 1737458799.271706
From The Process 1737458799.4344413
From The Process 1737458799.5139449
From The Process 1737458799.575038
From The Process 1737458799.7472038
From The Process 1737458799.8056705
From The Process 1737458799.9410806
From The Process 1737458800.2993653
From The Process 1737458800.3043957
From The Process 1737458800.5507133
From The Process 1737458800.5777469
From The Process 1737458800.6267283
From The Process 1737458800.7753623
From The Process 1737458801.0272012
Parallel processing of files fi

## First approach: send the whole data array to each child process

The slowest so far because the size of the data array is ~400 MB and it presumably takes a lot of time
to send a copy of the array to each child process.

In [7]:
""" #TODO

"""

import glob
import multiprocessing
import os
import sys
import time
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from daplis.functions import calc_diff as cd
from daplis.functions import utils
from daplis.functions.calibrate import load_calibration_data
from numpy import ndarray
from pyarrow import feather as ft


class MpWizard:

    # Initialize by passing the input parameters which later will be
    # passed into all internal functions
    def __init__(
        self,
        path: str = "",
        pixels: list = [],
        daughterboard_number: str = "",
        motherboard_number: str = "",
        firmware_version: str = "",
        timestamps: int = 512,
        delta_window: float = 50e3,
        include_offset: bool = False,
        apply_calibration: bool = True,
        apply_mask: bool = True,
        absolute_timestamps: bool = False,
        number_of_cores: int = 1,
    ):
        """Initialization of the class.

        Set all the input parameters for later use with internal class
        functions. Additionally, preload the calibration matrix.

        Parameters
        ----------
        path : str, optional
            _description_, by default ""
        pixels : list, optional
            _description_, by default []
        daughterboard_number : str, optional
            _description_, by default ""
        motherboard_number : str, optional
            _description_, by default ""
        firmware_version : str, optional
            _description_, by default ""
        timestamps : int, optional
            _description_, by default 512
        delta_window : float, optional
            _description_, by default 50e3
        include_offset : bool, optional
            _description_, by default False
        apply_calibration : bool, optional
            _description_, by default True
        apply_mask : bool, optional
            _description_, by default True
        absolute_timestamps : bool, optional
            _description_, by default False
        number_of_cores : int, optional
            _description_, by default 1
        """

        self.path = path
        self.pixels = pixels
        self.daughterboard_number = daughterboard_number
        self.motherboard_number = motherboard_number
        self.firmware_version = firmware_version
        self.timestamps = timestamps
        self.delta_window = delta_window
        self.include_offset = include_offset
        self.apply_calibration = apply_calibration
        self.apply_mask = apply_mask
        self.absolute_timestamps = absolute_timestamps
        self.number_of_cores = number_of_cores

        os.chdir(self.path)

        # Load calibration if requested
        if self.apply_calibration:

            # work_dir = Path(__file__).resolve().parent.parent

            # TODO
            # path_calibration_data = os.path.join(
            #     work_dir, r"params\calibration_data"
            # )
            path_calibration_data = (
                r"/home/sj/GitHub/daplis/src/daplis/params/calibration_data"
            )
            # path_calibration_data = r"C:\Users\bruce\Documents\GitHub\daplis\src\daplis\params\calibration_data"

            calibration_data = load_calibration_data(
                path_calibration_data,
                daughterboard_number,
                motherboard_number,
                firmware_version,
                include_offset,
            )

            if self.include_offset:
                self.calibration_matrix, self.offset_array = calibration_data
            else:
                self.calibration_matrix = calibration_data

        # Apply mask if requested
        if self.apply_mask:
            mask = utils.apply_mask(
                self.daughterboard_number,
                self.motherboard_number,
            )
            if isinstance(self.pixels[0], int) and isinstance(
                self.pixels[1], int
            ):
                self.pixels = [pix for pix in self.pixels if pix not in mask]
            else:
                self.pixels = [
                    [value for value in sublist if value not in mask]
                    for sublist in pixels
                ]

        # Check the firmware version and set the pixel coordinates accordingly
        if self.firmware_version == "2212s":
            self.pix_coor = np.arange(256).reshape(4, 64).T
        elif firmware_version == "2212b":
            self.pix_coor = np.arange(256).reshape(64, 4)
        else:
            print("\nFirmware version is not recognized.")
            sys.exit()

    def _unpack_binary_data(
        self,
        file: str,
    ) -> np.ndarray:
        """Unpack binary data from LinoSPAD2.

        Same unpacking function as the standard 'daplis' one, except
        the calibration matrix is preloaded during initialization and
        called here. Return a 3D matrix of pixel numbers and timestamps.

        Parameters
        ----------
        file : str
            A '.dat' data file from LinoSPAD2 with the binary-encoded
            data.

        Returns
        -------
        np.ndarray
            A 3D matrix of the pixel numbers where timestamp was
            recorded and timestamps themselves.
        """
        # Unpack binary data
        raw_data = np.memmap(file, dtype=np.uint32)
        # Timestamps are stored in the lower 28 bits
        data_timestamps = (raw_data & 0xFFFFFFF).astype(np.int64)
        # Pixel address in the given TDC is 2 bits above timestamp
        data_pixels = ((raw_data >> 28) & 0x3).astype(np.int8)
        # Check the top bit, assign '-1' to invalid timestamps
        data_timestamps[raw_data < 0x80000000] = -1

        # Number of acquisition cycles in each data file
        cycles = len(data_timestamps) // (self.timestamps * 65)
        # Transform into a matrix of size 65 by cycles*timestamps
        data_pixels = (
            data_pixels.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        data_timestamps = (
            data_timestamps.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        # Cut the 65th TDC that does not hold any actual data from pixels
        data_pixels = data_pixels[:-1]
        data_timestamps = data_timestamps[:-1]

        # Insert '-2' at the end of each cycle
        insert_indices = np.linspace(
            self.timestamps, cycles * self.timestamps, cycles
        ).astype(np.int64)

        data_pixels = np.insert(
            data_pixels,
            insert_indices,
            -2,
            1,
        )
        data_timestamps = np.insert(
            data_timestamps,
            insert_indices,
            -2,
            1,
        )

        # Combine both matrices into a single one, where each cell holds pixel
        # coordinates in the TDC and the timestamp
        data_all = np.stack((data_pixels, data_timestamps), axis=2).astype(
            np.int64
        )

        if self.apply_calibration is False:
            data_all[:, :, 1] = data_all[:, :, 1] * 2500 / 140
        else:
            # Path to the calibration data
            pix_coordinates = np.arange(256).reshape(64, 4)
            for i in range(256):
                # Transform pixel number to TDC number and pixel
                # coordinates in that TDC (from 0 to 3)
                tdc, pix = np.argwhere(pix_coordinates == i)[0]
                # Find data from that pixel
                ind = np.where(data_all[tdc].T[0] == pix)[0]
                # Cut non-valid timestamps ('-1's)
                ind = ind[data_all[tdc].T[1][ind] >= 0]
                if not np.any(ind):
                    continue
                data_cut = data_all[tdc].T[1][ind]
                # Apply calibration; offset is added due to how delta
                # ts are calculated
                if self.include_offset:
                    data_all[tdc].T[1][ind] = (
                        (data_cut - data_cut % 140) * 2500 / 140
                        + self.calibration_matrix[i, (data_cut % 140)]
                        + self.offset_array[i]
                    )
                else:
                    data_all[tdc].T[1][ind] = (
                        data_cut - data_cut % 140
                    ) * 2500 / 140 + self.calibration_matrix[
                        i, (data_cut % 140)
                    ]

        return data_all

    def _calculate_differences_2212_fast(
        self,
        data: ndarray,
        delta_window: float = 50e3,
        cycle_length: float = 4e9,
    ):
        """Calculate timestamp differences for firmware version 2212.

        Calculate timestamp differences for the given pixels and LinoSPAD2
        firmware version 2212. Modified compared to the standard 'daplis'
        one. Modifications are for working with prechunked data, i.e.,
        sliced down to two arrays from the whole 64xN matrix.

        Parameters
        ----------
        data : ndarray
            Matrix of timestamps, where rows correspond to the TDCs.
        pixels : List[int] | List[List[int]]
            List of pixel numbers for which the timestamp differences should
            be calculated or list of two lists with pixel numbers for peak
            vs. peak calculations.
        pix_coor : ndarray
            Array for transforming the pixel address in terms of TDC (0 to 3)
            to pixel number in terms of half of the sensor (0 to 255).
        delta_window : float, optional
            Width of the time window for counting timestamp differences.
            The default is 50e3 (50 ns).
        cycle_length : float, optional
            Length of each acquisition cycle. The default is 4e9 (4 ms).

        Returns
        -------
        deltas_all : dict
            Dictionary containing timestamp differences for each pair of pixels.

        """

        # Dictionary for the timestamp differences, where keys are the
        # pixel numbers of the requested pairs

        deltas_all = []

        data_pix_1 = data[0]
        data_pix_2 = data[1]

        indices1 = data_pix_1[0]
        indices2 = data_pix_2[0]
        timestamps1 = data_pix_1[1]
        timestamps2 = data_pix_2[1]

        timestamps_1 = []
        timestamps_2 = []

        # Go over cycles, shifting the timestamps from each next
        # cycle by lengths of cycles before (e.g., for the 4th cycle
        # add 12 ms)
        for i, _ in enumerate(self.cycle_ends[:-1]):
            slice_from = self.cycle_ends[i]
            slice_to = self.cycle_ends[i + 1]
            pix1_slice = indices1[
                (indices1 >= slice_from) & (indices1 < slice_to)
            ]
            if not np.any(pix1_slice):
                continue
            pix2_slice = indices2[
                (indices2 >= slice_from) & (indices2 < slice_to)
            ]
            if not np.any(pix2_slice):
                continue

            # Shift timestamps by cycle length
            tmsp1 = timestamps1[np.isin(indices1, pix1_slice)]
            tmsp1 = tmsp1[tmsp1 > 0]
            tmsp1 = tmsp1 + cycle_length * i

            tmsp2 = timestamps2[np.isin(indices2, pix2_slice)]
            tmsp2 = tmsp2[tmsp2 > 0]
            tmsp2 = tmsp2 + cycle_length * i

            timestamps_1.extend(tmsp1)
            timestamps_2.extend(tmsp2)

        timestamps_1 = np.array(timestamps_1)
        timestamps_2 = np.array(timestamps_2)

        # Indicators for each pixel: 0 for timestamps from one pixel
        # 1 - from the other
        pix1_ind = np.zeros(len(timestamps_1), dtype=np.int32)
        pix2_ind = np.ones(len(timestamps_2), dtype=np.int32)

        pix1_data = np.vstack((pix1_ind, timestamps_1))
        pix2_data = np.vstack((pix2_ind, timestamps_2))

        # Dataframe for each pixel with pixel indicator and
        # timestamps
        df1 = pd.DataFrame(pix1_data.T, columns=["Pixel_index", "Timestamp"])
        df2 = pd.DataFrame(pix2_data.T, columns=["Pixel_index", "Timestamp"])

        # Combine the two dataframes
        df_combined = pd.concat((df1, df2), ignore_index=True)

        # Sort the timestamps
        df_combined.sort_values("Timestamp", inplace=True)

        # Subtract pixel indicators of neighbors; values of 0
        # correspond to timestamp differences for the same pixel
        # '-1' and '1' - to differences from different pixels
        df_combined["Pixel_index_diff"] = df_combined["Pixel_index"].diff()

        # Calculate timestamp difference between neighbors
        df_combined["Timestamp_diff"] = df_combined["Timestamp"].diff()

        # Get the correct timestamp difference sign
        df_combined["Timestamp_diff"] = (
            df_combined["Timestamp_diff"] * df_combined["Pixel_index_diff"]
        )

        # Collect timestamp differences where timestamps are from
        # different pixels
        filtered_df = df_combined[abs(df_combined["Pixel_index_diff"]) == 1]

        # Save only timestamps differences in the requested window
        delta_ts = filtered_df[
            abs(filtered_df["Timestamp_diff"]) < delta_window
        ]["Timestamp_diff"].values

        deltas_all.extend(delta_ts)

        return deltas_all

    def _calculate_timestamps_differences(self, args):
        """Calculate photon coincidences and save to '.feather'.

        Parameters
        ----------
        #TODO
        """

        print("From The Process", time.time())

        # try-except railguard for a function that goes to separate
        # cores
        file, data, pixel_pair = args
        try:
            # Check if the 'delta_ts_data' folder exists
            output_dir = Path(self.path) / "delta_ts_data"
            output_dir.mkdir(exist_ok=True)

            # Calculate the differences and convert them to a pandas
            # dataframe - standard approach sending the whole matrix of
            # 64xN to each child process
            deltas_all = cd.calculate_differences_2212_fast(
                data, pixel_pair, self.pix_coor
            )

            # Calculate the differences and convert them to a pandas
            # dataframe - pre-chunking the data down to 2 arrays
            # deltas_all = self._calculate_differences_2212_fast(data)
            
            data_for_plot_df = pd.DataFrame(
                deltas_all, columns=[f"{pixel_pair[0]},{pixel_pair[1]}"]
            ).T

            # Save the data to a '.feather' file
            file_name = Path(
                file
            ).stem  # Get the file name without the extension
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            file_name = Path(file).stem
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            ft.write_feather(
                data_for_plot_df.reset_index(drop=True), output_file
            )

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    def _combine_feather_files(self, path_to_feather_files: str):

        os.chdir(path_to_feather_files)

        for pixel_pair in self.pixel_pairs:
            ft_files = glob.glob(f"*{pixel_pair[0]}_{pixel_pair[1]}*.feather")
            data_all = pd.DataFrame()
            for ft_file in ft_files:
                data = ft.read_feather(ft_file)
                data_all = pd.concat((data_all, data), ignore_index=True)
            data_all.to_feather(
                f"combined_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )

    ### Heavy data chunking - fast calculation in each child process at
    ### the cost of long chunking - no acceleration as a result
    # def _chunk_that_data_too_much(self, data, pixel_pair, cycle_length: float = 4e9):

    #     pix_coor = np.arange(256).reshape(64, 4)

    #     # Find ends of cycles
    #     cycle_ends = np.argwhere(data[0].T[0] == -2)
    #     cycle_ends = np.insert(cycle_ends, 0, 0)

    #     tdc1, pix_c1 = np.argwhere(pix_coor == pixel_pair[0])[0]
    #     pix1 = np.where(data[tdc1].T[0] == pix_c1)[0]

    #     timestamps_1 = []
    #     timestamps_2 = []

    #     # Second pixel in the pair
    #     tdc2, pix_c2 = np.argwhere(pix_coor == pixel_pair[1])[0]
    #     pix2 = np.where(data[tdc2].T[0] == pix_c2)[0]

    #     # Go over cycles, shifting the timestamps from each next
    #     # cycle by lengths of cycles before (e.g., for the 4th cycle
    #     # add 12 ms)
    #     for i, _ in enumerate(cycle_ends[:-1]):
    #         slice_from = cycle_ends[i]
    #         slice_to = cycle_ends[i + 1]
    #         pix1_slice = pix1[(pix1 >= slice_from) & (pix1 < slice_to)]
    #         if not np.any(pix1_slice):
    #             continue
    #         pix2_slice = pix2[(pix2 >= slice_from) & (pix2 < slice_to)]
    #         if not np.any(pix2_slice):
    #             continue

    #         # Shift timestamps by cycle length
    #         tmsp1 = data[tdc1].T[1][pix1_slice]
    #         tmsp1 = tmsp1[tmsp1 > 0]
    #         tmsp1 = tmsp1 + cycle_length * i

    #         tmsp2 = data[tdc2].T[1][pix2_slice]
    #         tmsp2 = tmsp2[tmsp2 > 0]
    #         tmsp2 = tmsp2 + cycle_length * i

    #         timestamps_1.extend(tmsp1)
    #         timestamps_2.extend(tmsp2)

    #     timestamps_1 = np.array(timestamps_1)
    #     timestamps_2 = np.array(timestamps_2)

    #     return (timestamps_1, timestamps_2)

    def _chunk_that_data(self, data, pixel_pair, cycle_length: float = 4e9):
        """Chunk data down to 2 rows - prepare for child processes.

        Parameters
        ----------
        data : np.ndarray
            The whole matrix.
        pixel_pair : list
            Pair of pixels to slice out of the whole matrix.
        cycle_length : float, optional
            Cycle length in ps, by default 4e9

        Returns
        -------
        tuple
            Tuple of two arrays, each contains timestamps from the two
            pixels requested and the indices of the timestamps in the
            original matrix of data. The indices are used for correct
            assigning to the corresponding acquisition cycles.
        """

        tdc1, pix_c1 = np.argwhere(self.pix_coor == pixel_pair[0])[0]
        indices1 = np.where(data[tdc1].T[0] == pix_c1)[0]

        # Second pixel in the pair
        tdc2, pix_c2 = np.argwhere(self.pix_coor == pixel_pair[1])[0]
        indices2 = np.where(data[tdc2].T[0] == pix_c2)[0]

        data_cut_1 = np.array((indices1, data[tdc1].T[1][indices1]))
        data_cut_2 = np.array((indices2, data[tdc2].T[1][indices2]))

        return (data_cut_1, data_cut_2)

    def calculate_and_save_timestamp_differences_mp(self):

        # Find all LinoSPAD2 data files
        files = glob.glob("*.dat")

        if not files:
            raise ValueError("No .dat files found in the specified path.")

        self.pixel_pairs = []
        for i in self.pixels[0]:
            for j in self.pixels[1]:
                self.pixel_pairs.append([i, j])

        start_time = time.time()

        # Go file by file
        for file in files:
            print(f"Processing file: {file}")

            start_time_unpack = time.time()

            # Unpack the data from the file
            data = self._unpack_binary_data(file)

            # Pre-collect the indices of the acquisition cycles' ends
            self.cycle_ends = np.argwhere(data[0].T[0] == -2)
            self.cycle_ends = np.insert(self.cycle_ends, 0, 0)

            start_time_args = time.time()

            # Prepare the arguments for the child processes:
            # 'file' and 'pixel_pair' for naming purposes,
            # chunked data as a tuple of two arrays to work with
            args = [
                (
                    file,
                    data,
                    pixel_pair,
                )
                for pixel_pair in self.pixel_pairs
            ]

            # print(f"Created args in {time.time() - start_time_args}")
            print("Before the processes", time.time())

            with multiprocessing.Pool(
                min(self.number_of_cores, len(self.pixel_pairs))
            ) as pool:
                pool.map(
                    self._calculate_timestamps_differences,
                    args,
                )

        end_time = time.time()

        print(
            f"Parallel processing of files "
            "files (with each writing to its file) finished "
            f"in: {round(end_time - start_time, 2)} s"
        )

        # Combine '.feather' files from separate cores
        path_to_feathers = os.path.join(self.path, "delta_ts_data")

        self._combine_feather_files(path_to_feathers)

        path_output = os.path.join(self.path, "delta_ts_data")

        print(
            "The feather files with the timestamp differences were "
            f"combined into the 'combined.feather' file in {path_output}"
        )


if __name__ == "__main__":
    # path = r"D:\LinoSPAD2\Data\B7d\2024.11.11\500t\MP_test"
    path = r'/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70'

    mp = MpWizard(
        path,
        # pixels=[[144], [171, 172]],
        pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
        # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
        daughterboard_number="B7d",
        motherboard_number="#28",
        firmware_version="2212b",
        timestamps=500,
        number_of_cores=5,
    )

    mp.calculate_and_save_timestamp_differences_mp()


### Standard approach - for control and comparison

# import time

# from daplis.functions import delta_t

# time_start = time.time()

# path = r"/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70"


# delta_t.calculate_and_save_timestamp_differences_fast(
#     path,
#     rewrite=True,
#     # pixels=[144, 171],
#     pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
#     # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
#     daughterboard_number="B7d",
#     motherboard_number="#28",
#     firmware_version="2212b",
#     timestamps=500,
# )

# print(f"Finished in {time.time() - time_start}")

Processing file: 0000054222.dat
Before the processes 1737458942.433863
From The Process 1737458943.4432573
From The Process 1737458943.7281208
From The Process 1737458944.2579844
From The Process 1737458944.5373874
From The Process 1737458945.0620754
From The Process 1737458945.3460772
From The Process 1737458945.875059
From The Process 1737458946.3766
From The Process 1737458946.6815412
From The Process 1737458947.0119455
From The Process 1737458947.4802182
From The Process 1737458947.8315656
From The Process 1737458948.2624516
From The Process 1737458948.6082184
From The Process 1737458949.0463061
From The Process 1737458949.3998938
From The Process 1737458949.8204172
From The Process 1737458950.3264365
From The Process 1737458950.5878406
From The Process 1737458950.9382226
From The Process 1737458951.3753674
From The Process 1737458951.7286372
From The Process 1737458952.1567428
From The Process 1737458952.509657
From The Process 1737458952.9016325
Parallel processing of files files

## Second approach: preprocess the data before sending to each child process

Faster but still not good. Each child process works very fast but preparing the data takes a lot of time.

In [10]:
""" #TODO

"""

import glob
import multiprocessing
import os
import sys
import time
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from daplis.functions import calc_diff as cd
from daplis.functions import utils
from daplis.functions.calibrate import load_calibration_data
from numpy import ndarray
from pyarrow import feather as ft


class MpWizard:

    # Initialize by passing the input parameters which later will be
    # passed into all internal functions
    def __init__(
        self,
        path: str = "",
        pixels: list = [],
        daughterboard_number: str = "",
        motherboard_number: str = "",
        firmware_version: str = "",
        timestamps: int = 512,
        delta_window: float = 50e3,
        include_offset: bool = False,
        apply_calibration: bool = True,
        apply_mask: bool = True,
        absolute_timestamps: bool = False,
        number_of_cores: int = 1,
    ):
        """Initialization of the class.

        Set all the input parameters for later use with internal class
        functions. Additionally, preload the calibration matrix.

        Parameters
        ----------
        path : str, optional
            _description_, by default ""
        pixels : list, optional
            _description_, by default []
        daughterboard_number : str, optional
            _description_, by default ""
        motherboard_number : str, optional
            _description_, by default ""
        firmware_version : str, optional
            _description_, by default ""
        timestamps : int, optional
            _description_, by default 512
        delta_window : float, optional
            _description_, by default 50e3
        include_offset : bool, optional
            _description_, by default False
        apply_calibration : bool, optional
            _description_, by default True
        apply_mask : bool, optional
            _description_, by default True
        absolute_timestamps : bool, optional
            _description_, by default False
        number_of_cores : int, optional
            _description_, by default 1
        """

        self.path = path
        self.pixels = pixels
        self.daughterboard_number = daughterboard_number
        self.motherboard_number = motherboard_number
        self.firmware_version = firmware_version
        self.timestamps = timestamps
        self.delta_window = delta_window
        self.include_offset = include_offset
        self.apply_calibration = apply_calibration
        self.apply_mask = apply_mask
        self.absolute_timestamps = absolute_timestamps
        self.number_of_cores = number_of_cores

        os.chdir(self.path)

        # Load calibration if requested
        if self.apply_calibration:

            # work_dir = Path(__file__).resolve().parent.parent

            # TODO
            # path_calibration_data = os.path.join(
            #     work_dir, r"params\calibration_data"
            # )
            path_calibration_data = (
                r"/home/sj/GitHub/daplis/src/daplis/params/calibration_data"
            )
            # path_calibration_data = r"C:\Users\bruce\Documents\GitHub\daplis\src\daplis\params\calibration_data"

            calibration_data = load_calibration_data(
                path_calibration_data,
                daughterboard_number,
                motherboard_number,
                firmware_version,
                include_offset,
            )

            if self.include_offset:
                self.calibration_matrix, self.offset_array = calibration_data
            else:
                self.calibration_matrix = calibration_data

        # Apply mask if requested
        if self.apply_mask:
            mask = utils.apply_mask(
                self.daughterboard_number,
                self.motherboard_number,
            )
            if isinstance(self.pixels[0], int) and isinstance(
                self.pixels[1], int
            ):
                self.pixels = [pix for pix in self.pixels if pix not in mask]
            else:
                self.pixels = [
                    [value for value in sublist if value not in mask]
                    for sublist in pixels
                ]

        # Check the firmware version and set the pixel coordinates accordingly
        if self.firmware_version == "2212s":
            self.pix_coor = np.arange(256).reshape(4, 64).T
        elif firmware_version == "2212b":
            self.pix_coor = np.arange(256).reshape(64, 4)
        else:
            print("\nFirmware version is not recognized.")
            sys.exit()

    def _unpack_binary_data(
        self,
        file: str,
    ) -> np.ndarray:
        """Unpack binary data from LinoSPAD2.

        Same unpacking function as the standard 'daplis' one, except
        the calibration matrix is preloaded during initialization and
        called here. Return a 3D matrix of pixel numbers and timestamps.

        Parameters
        ----------
        file : str
            A '.dat' data file from LinoSPAD2 with the binary-encoded
            data.

        Returns
        -------
        np.ndarray
            A 3D matrix of the pixel numbers where timestamp was
            recorded and timestamps themselves.
        """
        # Unpack binary data
        raw_data = np.memmap(file, dtype=np.uint32)
        # Timestamps are stored in the lower 28 bits
        data_timestamps = (raw_data & 0xFFFFFFF).astype(np.int64)
        # Pixel address in the given TDC is 2 bits above timestamp
        data_pixels = ((raw_data >> 28) & 0x3).astype(np.int8)
        # Check the top bit, assign '-1' to invalid timestamps
        data_timestamps[raw_data < 0x80000000] = -1

        # Number of acquisition cycles in each data file
        cycles = len(data_timestamps) // (self.timestamps * 65)
        # Transform into a matrix of size 65 by cycles*timestamps
        data_pixels = (
            data_pixels.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        data_timestamps = (
            data_timestamps.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        # Cut the 65th TDC that does not hold any actual data from pixels
        data_pixels = data_pixels[:-1]
        data_timestamps = data_timestamps[:-1]

        # Insert '-2' at the end of each cycle
        insert_indices = np.linspace(
            self.timestamps, cycles * self.timestamps, cycles
        ).astype(np.int64)

        data_pixels = np.insert(
            data_pixels,
            insert_indices,
            -2,
            1,
        )
        data_timestamps = np.insert(
            data_timestamps,
            insert_indices,
            -2,
            1,
        )

        # Combine both matrices into a single one, where each cell holds pixel
        # coordinates in the TDC and the timestamp
        data_all = np.stack((data_pixels, data_timestamps), axis=2).astype(
            np.int64
        )

        if self.apply_calibration is False:
            data_all[:, :, 1] = data_all[:, :, 1] * 2500 / 140
        else:
            # Path to the calibration data
            pix_coordinates = np.arange(256).reshape(64, 4)
            for i in range(256):
                # Transform pixel number to TDC number and pixel
                # coordinates in that TDC (from 0 to 3)
                tdc, pix = np.argwhere(pix_coordinates == i)[0]
                # Find data from that pixel
                ind = np.where(data_all[tdc].T[0] == pix)[0]
                # Cut non-valid timestamps ('-1's)
                ind = ind[data_all[tdc].T[1][ind] >= 0]
                if not np.any(ind):
                    continue
                data_cut = data_all[tdc].T[1][ind]
                # Apply calibration; offset is added due to how delta
                # ts are calculated
                if self.include_offset:
                    data_all[tdc].T[1][ind] = (
                        (data_cut - data_cut % 140) * 2500 / 140
                        + self.calibration_matrix[i, (data_cut % 140)]
                        + self.offset_array[i]
                    )
                else:
                    data_all[tdc].T[1][ind] = (
                        data_cut - data_cut % 140
                    ) * 2500 / 140 + self.calibration_matrix[
                        i, (data_cut % 140)
                    ]

        return data_all

    def _calculate_differences_2212_fast(
        self,
        data: ndarray,
        delta_window: float = 50e3,
        cycle_length: float = 4e9,
    ):
        """Calculate timestamp differences for firmware version 2212.

        Calculate timestamp differences for the given pixels and LinoSPAD2
        firmware version 2212. Modified compared to the standard 'daplis'
        one. Modifications are for working with prechunked data, i.e.,
        sliced down to two arrays from the whole 64xN matrix.

        Parameters
        ----------
        data : ndarray
            Matrix of timestamps, where rows correspond to the TDCs.
        pixels : List[int] | List[List[int]]
            List of pixel numbers for which the timestamp differences should
            be calculated or list of two lists with pixel numbers for peak
            vs. peak calculations.
        pix_coor : ndarray
            Array for transforming the pixel address in terms of TDC (0 to 3)
            to pixel number in terms of half of the sensor (0 to 255).
        delta_window : float, optional
            Width of the time window for counting timestamp differences.
            The default is 50e3 (50 ns).
        cycle_length : float, optional
            Length of each acquisition cycle. The default is 4e9 (4 ms).

        Returns
        -------
        deltas_all : dict
            Dictionary containing timestamp differences for each pair of pixels.

        """

        # Dictionary for the timestamp differences, where keys are the
        # pixel numbers of the requested pairs

        deltas_all = []

        # data_pix_1 = data[0]
        # data_pix_2 = data[1]

        # indices1 = data_pix_1[0]
        # indices2 = data_pix_2[0]
        # timestamps1 = data_pix_1[1]
        # timestamps2 = data_pix_2[1]

        # timestamps_1 = []
        # timestamps_2 = []

        # # Go over cycles, shifting the timestamps from each next
        # # cycle by lengths of cycles before (e.g., for the 4th cycle
        # # add 12 ms)
        # for i, _ in enumerate(self.cycle_ends[:-1]):
        #     slice_from = self.cycle_ends[i]
        #     slice_to = self.cycle_ends[i + 1]
        #     pix1_slice = indices1[
        #         (indices1 >= slice_from) & (indices1 < slice_to)
        #     ]
        #     if not np.any(pix1_slice):
        #         continue
        #     pix2_slice = indices2[
        #         (indices2 >= slice_from) & (indices2 < slice_to)
        #     ]
        #     if not np.any(pix2_slice):
        #         continue

        #     # Shift timestamps by cycle length
        #     tmsp1 = timestamps1[np.isin(indices1, pix1_slice)]
        #     tmsp1 = tmsp1[tmsp1 > 0]
        #     tmsp1 = tmsp1 + cycle_length * i

        #     tmsp2 = timestamps2[np.isin(indices2, pix2_slice)]
        #     tmsp2 = tmsp2[tmsp2 > 0]
        #     tmsp2 = tmsp2 + cycle_length * i

        #     timestamps_1.extend(tmsp1)
        #     timestamps_2.extend(tmsp2)

        timestamps_1 = data[0]
        timestamps_2 = data[1]

        # Indicators for each pixel: 0 for timestamps from one pixel
        # 1 - from the other
        pix1_ind = np.zeros(len(timestamps_1), dtype=np.int32)
        pix2_ind = np.ones(len(timestamps_2), dtype=np.int32)

        pix1_data = np.vstack((pix1_ind, timestamps_1))
        pix2_data = np.vstack((pix2_ind, timestamps_2))

        # Dataframe for each pixel with pixel indicator and
        # timestamps
        df1 = pd.DataFrame(pix1_data.T, columns=["Pixel_index", "Timestamp"])
        df2 = pd.DataFrame(pix2_data.T, columns=["Pixel_index", "Timestamp"])

        # Combine the two dataframes
        df_combined = pd.concat((df1, df2), ignore_index=True)

        # Sort the timestamps
        df_combined.sort_values("Timestamp", inplace=True)

        # Subtract pixel indicators of neighbors; values of 0
        # correspond to timestamp differences for the same pixel
        # '-1' and '1' - to differences from different pixels
        df_combined["Pixel_index_diff"] = df_combined["Pixel_index"].diff()

        # Calculate timestamp difference between neighbors
        df_combined["Timestamp_diff"] = df_combined["Timestamp"].diff()

        # Get the correct timestamp difference sign
        df_combined["Timestamp_diff"] = (
            df_combined["Timestamp_diff"] * df_combined["Pixel_index_diff"]
        )

        # Collect timestamp differences where timestamps are from
        # different pixels
        filtered_df = df_combined[abs(df_combined["Pixel_index_diff"]) == 1]

        # Save only timestamps differences in the requested window
        delta_ts = filtered_df[
            abs(filtered_df["Timestamp_diff"]) < delta_window
        ]["Timestamp_diff"].values

        deltas_all.extend(delta_ts)

        return deltas_all

    def _calculate_timestamps_differences(self, args):
        """Calculate photon coincidences and save to '.feather'.

        Parameters
        ----------
        #TODO
        """

        print("From The Process", time.time())

        # try-except railguard for a function that goes to separate
        # cores
        file, data, pixel_pair = args
        try:
            # Check if the 'delta_ts_data' folder exists
            output_dir = Path(self.path) / "delta_ts_data"
            output_dir.mkdir(exist_ok=True)

            # Calculate the differences and convert them to a pandas
            # dataframe - standard approach sending the whole matrix of
            # 64xN to each child process
            # deltas_all = cd.calculate_differences_2212_fast(
            #     data, pixel_pair, self.pix_coor
            # )

            # Calculate the differences and convert them to a pandas
            # dataframe - pre-chunking the data down to 2 arrays
            deltas_all = self._calculate_differences_2212_fast(data)
            
            data_for_plot_df = pd.DataFrame(
                deltas_all, columns=[f"{pixel_pair[0]},{pixel_pair[1]}"]
            ).T

            # Save the data to a '.feather' file
            file_name = Path(
                file
            ).stem  # Get the file name without the extension
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            file_name = Path(file).stem
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            ft.write_feather(
                data_for_plot_df.reset_index(drop=True), output_file
            )

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    def _combine_feather_files(self, path_to_feather_files: str):

        os.chdir(path_to_feather_files)

        for pixel_pair in self.pixel_pairs:
            ft_files = glob.glob(f"*{pixel_pair[0]}_{pixel_pair[1]}*.feather")
            data_all = pd.DataFrame()
            for ft_file in ft_files:
                data = ft.read_feather(ft_file)
                data_all = pd.concat((data_all, data), ignore_index=True)
            data_all.to_feather(
                f"combined_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )

    ## Heavy data chunking - fast calculation in each child process at
    ## the cost of long chunking - no acceleration as a result
    def _chunk_that_data_a_lot(self, data, pixel_pair, cycle_length: float = 4e9):

        pix_coor = np.arange(256).reshape(64, 4)

        # Find ends of cycles
        cycle_ends = np.argwhere(data[0].T[0] == -2)
        cycle_ends = np.insert(cycle_ends, 0, 0)

        tdc1, pix_c1 = np.argwhere(pix_coor == pixel_pair[0])[0]
        pix1 = np.where(data[tdc1].T[0] == pix_c1)[0]

        timestamps_1 = []
        timestamps_2 = []

        # Second pixel in the pair
        tdc2, pix_c2 = np.argwhere(pix_coor == pixel_pair[1])[0]
        pix2 = np.where(data[tdc2].T[0] == pix_c2)[0]

        # Go over cycles, shifting the timestamps from each next
        # cycle by lengths of cycles before (e.g., for the 4th cycle
        # add 12 ms)
        for i, _ in enumerate(cycle_ends[:-1]):
            slice_from = cycle_ends[i]
            slice_to = cycle_ends[i + 1]
            pix1_slice = pix1[(pix1 >= slice_from) & (pix1 < slice_to)]
            if not np.any(pix1_slice):
                continue
            pix2_slice = pix2[(pix2 >= slice_from) & (pix2 < slice_to)]
            if not np.any(pix2_slice):
                continue

            # Shift timestamps by cycle length
            tmsp1 = data[tdc1].T[1][pix1_slice]
            tmsp1 = tmsp1[tmsp1 > 0]
            tmsp1 = tmsp1 + cycle_length * i

            tmsp2 = data[tdc2].T[1][pix2_slice]
            tmsp2 = tmsp2[tmsp2 > 0]
            tmsp2 = tmsp2 + cycle_length * i

            timestamps_1.extend(tmsp1)
            timestamps_2.extend(tmsp2)

        timestamps_1 = np.array(timestamps_1)
        timestamps_2 = np.array(timestamps_2)

        return (timestamps_1, timestamps_2)

    # def _chunk_that_data(self, data, pixel_pair, cycle_length: float = 4e9):
    #     """Chunk data down to 2 rows - prepare for child processes.

    #     Parameters
    #     ----------
    #     data : np.ndarray
    #         The whole matrix.
    #     pixel_pair : list
    #         Pair of pixels to slice out of the whole matrix.
    #     cycle_length : float, optional
    #         Cycle length in ps, by default 4e9

    #     Returns
    #     -------
    #     tuple
    #         Tuple of two arrays, each contains timestamps from the two
    #         pixels requested and the indices of the timestamps in the
    #         original matrix of data. The indices are used for correct
    #         assigning to the corresponding acquisition cycles.
    #     """

    #     tdc1, pix_c1 = np.argwhere(self.pix_coor == pixel_pair[0])[0]
    #     indices1 = np.where(data[tdc1].T[0] == pix_c1)[0]

    #     # Second pixel in the pair
    #     tdc2, pix_c2 = np.argwhere(self.pix_coor == pixel_pair[1])[0]
    #     indices2 = np.where(data[tdc2].T[0] == pix_c2)[0]

    #     data_cut_1 = np.array((indices1, data[tdc1].T[1][indices1]))
    #     data_cut_2 = np.array((indices2, data[tdc2].T[1][indices2]))

    #     return (data_cut_1, data_cut_2)

    def calculate_and_save_timestamp_differences_mp(self):

        # Find all LinoSPAD2 data files
        files = glob.glob("*.dat")

        if not files:
            raise ValueError("No .dat files found in the specified path.")

        self.pixel_pairs = []
        for i in self.pixels[0]:
            for j in self.pixels[1]:
                self.pixel_pairs.append([i, j])

        start_time = time.time()

        # Go file by file
        for file in files:
            print(f"Processing file: {file}")

            start_time_unpack = time.time()

            # Unpack the data from the file
            data = self._unpack_binary_data(file)

            # Pre-collect the indices of the acquisition cycles' ends
            self.cycle_ends = np.argwhere(data[0].T[0] == -2)
            self.cycle_ends = np.insert(self.cycle_ends, 0, 0)

            start_time_args = time.time()

            # Prepare the arguments for the child processes:
            # 'file' and 'pixel_pair' for naming purposes,
            # chunked data as a tuple of two arrays to work with
            args = [
                (
                    file,
                    self._chunk_that_data_a_lot(data, pixel_pair),
                    pixel_pair,
                )
                for pixel_pair in self.pixel_pairs
            ]

            # print(f"Created args in {time.time() - start_time_args}")
            print("Before the processes", time.time())

            with multiprocessing.Pool(
                min(self.number_of_cores, len(self.pixel_pairs))
            ) as pool:
                pool.map(
                    self._calculate_timestamps_differences,
                    args,
                )

        end_time = time.time()

        print(
            f"Parallel processing of files "
            "files (with each writing to its file) finished "
            f"in: {round(end_time - start_time, 2)} s"
        )

        # Combine '.feather' files from separate cores
        path_to_feathers = os.path.join(self.path, "delta_ts_data")

        self._combine_feather_files(path_to_feathers)

        path_output = os.path.join(self.path, "delta_ts_data")

        print(
            "The feather files with the timestamp differences were "
            f"combined into the 'combined.feather' file in {path_output}"
        )


if __name__ == "__main__":
    # path = r"D:\LinoSPAD2\Data\B7d\2024.11.11\500t\MP_test"
    path = r'/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70'

    mp = MpWizard(
        path,
        # pixels=[[144], [171, 172]],
        pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
        # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
        daughterboard_number="B7d",
        motherboard_number="#28",
        firmware_version="2212b",
        timestamps=500,
        number_of_cores=5,
    )

    mp.calculate_and_save_timestamp_differences_mp()


### Standard approach - for control and comparison

# import time

# from daplis.functions import delta_t

# time_start = time.time()

# path = r"/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70"


# delta_t.calculate_and_save_timestamp_differences_fast(
#     path,
#     rewrite=True,
#     # pixels=[144, 171],
#     pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
#     # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
#     daughterboard_number="B7d",
#     motherboard_number="#28",
#     firmware_version="2212b",
#     timestamps=500,
# )

# print(f"Finished in {time.time() - time_start}")

Processing file: 0000054222.dat
Before the processes 1737459558.2638874
From The Process 1737459558.303448
From The Process 1737459558.3140216
From The Process 1737459558.3260505
From The Process 1737459558.341788
From The Process 1737459558.3604708
From The Process 1737459558.628499
From The Process 1737459558.6332514
From The Process From The Process1737459558.6536448
 From The Process1737459558.654796 
1737459558.6567192
From The Process 1737459558.753786
From The Process 1737459558.7629943
From The Process 1737459558.7910752
From The Process 1737459558.8094456
From The Process 1737459558.8205626
From The Process 1737459559.032863From The Process
 1737459559.0349278
From The Process 1737459559.06195
From The Process From The Process1737459559.0645025
 1737459559.065667
From The Process 1737459559.2807171
From The Process 1737459559.2867618
From The Process 1737459559.3289568
From The Process From The Process1737459559.4085212 
1737459559.409383
Parallel processing of files files (wi

## Third approach: prepare the data but not that much

Select the timestamps from the two requested pixels while keeping the indices of the timestamps, i.e., their position. Important for aligning the acquisition cycles.

Fastest yet but still on par with the standard sequential approach.

In [12]:
""" #TODO

"""

import glob
import multiprocessing
import os
import sys
import time
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from daplis.functions import calc_diff as cd
from daplis.functions import utils
from daplis.functions.calibrate import load_calibration_data
from numpy import ndarray
from pyarrow import feather as ft


class MpWizard:

    # Initialize by passing the input parameters which later will be
    # passed into all internal functions
    def __init__(
        self,
        path: str = "",
        pixels: list = [],
        daughterboard_number: str = "",
        motherboard_number: str = "",
        firmware_version: str = "",
        timestamps: int = 512,
        delta_window: float = 50e3,
        include_offset: bool = False,
        apply_calibration: bool = True,
        apply_mask: bool = True,
        absolute_timestamps: bool = False,
        number_of_cores: int = 1,
    ):
        """Initialization of the class.

        Set all the input parameters for later use with internal class
        functions. Additionally, preload the calibration matrix.

        Parameters
        ----------
        path : str, optional
            _description_, by default ""
        pixels : list, optional
            _description_, by default []
        daughterboard_number : str, optional
            _description_, by default ""
        motherboard_number : str, optional
            _description_, by default ""
        firmware_version : str, optional
            _description_, by default ""
        timestamps : int, optional
            _description_, by default 512
        delta_window : float, optional
            _description_, by default 50e3
        include_offset : bool, optional
            _description_, by default False
        apply_calibration : bool, optional
            _description_, by default True
        apply_mask : bool, optional
            _description_, by default True
        absolute_timestamps : bool, optional
            _description_, by default False
        number_of_cores : int, optional
            _description_, by default 1
        """

        self.path = path
        self.pixels = pixels
        self.daughterboard_number = daughterboard_number
        self.motherboard_number = motherboard_number
        self.firmware_version = firmware_version
        self.timestamps = timestamps
        self.delta_window = delta_window
        self.include_offset = include_offset
        self.apply_calibration = apply_calibration
        self.apply_mask = apply_mask
        self.absolute_timestamps = absolute_timestamps
        self.number_of_cores = number_of_cores

        os.chdir(self.path)

        # Load calibration if requested
        if self.apply_calibration:

            # work_dir = Path(__file__).resolve().parent.parent

            # TODO
            # path_calibration_data = os.path.join(
            #     work_dir, r"params\calibration_data"
            # )
            path_calibration_data = (
                r"/home/sj/GitHub/daplis/src/daplis/params/calibration_data"
            )
            # path_calibration_data = r"C:\Users\bruce\Documents\GitHub\daplis\src\daplis\params\calibration_data"

            calibration_data = load_calibration_data(
                path_calibration_data,
                daughterboard_number,
                motherboard_number,
                firmware_version,
                include_offset,
            )

            if self.include_offset:
                self.calibration_matrix, self.offset_array = calibration_data
            else:
                self.calibration_matrix = calibration_data

        # Apply mask if requested
        if self.apply_mask:
            mask = utils.apply_mask(
                self.daughterboard_number,
                self.motherboard_number,
            )
            if isinstance(self.pixels[0], int) and isinstance(
                self.pixels[1], int
            ):
                self.pixels = [pix for pix in self.pixels if pix not in mask]
            else:
                self.pixels = [
                    [value for value in sublist if value not in mask]
                    for sublist in pixels
                ]

        # Check the firmware version and set the pixel coordinates accordingly
        if self.firmware_version == "2212s":
            self.pix_coor = np.arange(256).reshape(4, 64).T
        elif firmware_version == "2212b":
            self.pix_coor = np.arange(256).reshape(64, 4)
        else:
            print("\nFirmware version is not recognized.")
            sys.exit()

    def _unpack_binary_data(
        self,
        file: str,
    ) -> np.ndarray:
        """Unpack binary data from LinoSPAD2.

        Same unpacking function as the standard 'daplis' one, except
        the calibration matrix is preloaded during initialization and
        called here. Return a 3D matrix of pixel numbers and timestamps.

        Parameters
        ----------
        file : str
            A '.dat' data file from LinoSPAD2 with the binary-encoded
            data.

        Returns
        -------
        np.ndarray
            A 3D matrix of the pixel numbers where timestamp was
            recorded and timestamps themselves.
        """
        # Unpack binary data
        raw_data = np.memmap(file, dtype=np.uint32)
        # Timestamps are stored in the lower 28 bits
        data_timestamps = (raw_data & 0xFFFFFFF).astype(np.int64)
        # Pixel address in the given TDC is 2 bits above timestamp
        data_pixels = ((raw_data >> 28) & 0x3).astype(np.int8)
        # Check the top bit, assign '-1' to invalid timestamps
        data_timestamps[raw_data < 0x80000000] = -1

        # Number of acquisition cycles in each data file
        cycles = len(data_timestamps) // (self.timestamps * 65)
        # Transform into a matrix of size 65 by cycles*timestamps
        data_pixels = (
            data_pixels.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        data_timestamps = (
            data_timestamps.reshape(cycles, 65, self.timestamps)
            .transpose((1, 0, 2))
            .reshape(65, -1)
        )

        # Cut the 65th TDC that does not hold any actual data from pixels
        data_pixels = data_pixels[:-1]
        data_timestamps = data_timestamps[:-1]

        # Insert '-2' at the end of each cycle
        insert_indices = np.linspace(
            self.timestamps, cycles * self.timestamps, cycles
        ).astype(np.int64)

        data_pixels = np.insert(
            data_pixels,
            insert_indices,
            -2,
            1,
        )
        data_timestamps = np.insert(
            data_timestamps,
            insert_indices,
            -2,
            1,
        )

        # Combine both matrices into a single one, where each cell holds pixel
        # coordinates in the TDC and the timestamp
        data_all = np.stack((data_pixels, data_timestamps), axis=2).astype(
            np.int64
        )

        if self.apply_calibration is False:
            data_all[:, :, 1] = data_all[:, :, 1] * 2500 / 140
        else:
            # Path to the calibration data
            pix_coordinates = np.arange(256).reshape(64, 4)
            for i in range(256):
                # Transform pixel number to TDC number and pixel
                # coordinates in that TDC (from 0 to 3)
                tdc, pix = np.argwhere(pix_coordinates == i)[0]
                # Find data from that pixel
                ind = np.where(data_all[tdc].T[0] == pix)[0]
                # Cut non-valid timestamps ('-1's)
                ind = ind[data_all[tdc].T[1][ind] >= 0]
                if not np.any(ind):
                    continue
                data_cut = data_all[tdc].T[1][ind]
                # Apply calibration; offset is added due to how delta
                # ts are calculated
                if self.include_offset:
                    data_all[tdc].T[1][ind] = (
                        (data_cut - data_cut % 140) * 2500 / 140
                        + self.calibration_matrix[i, (data_cut % 140)]
                        + self.offset_array[i]
                    )
                else:
                    data_all[tdc].T[1][ind] = (
                        data_cut - data_cut % 140
                    ) * 2500 / 140 + self.calibration_matrix[
                        i, (data_cut % 140)
                    ]

        return data_all

    def _calculate_differences_2212_fast(
        self,
        data: ndarray,
        delta_window: float = 50e3,
        cycle_length: float = 4e9,
    ):
        """Calculate timestamp differences for firmware version 2212.

        Calculate timestamp differences for the given pixels and LinoSPAD2
        firmware version 2212. Modified compared to the standard 'daplis'
        one. Modifications are for working with prechunked data, i.e.,
        sliced down to two arrays from the whole 64xN matrix.

        Parameters
        ----------
        data : ndarray
            Matrix of timestamps, where rows correspond to the TDCs.
        pixels : List[int] | List[List[int]]
            List of pixel numbers for which the timestamp differences should
            be calculated or list of two lists with pixel numbers for peak
            vs. peak calculations.
        pix_coor : ndarray
            Array for transforming the pixel address in terms of TDC (0 to 3)
            to pixel number in terms of half of the sensor (0 to 255).
        delta_window : float, optional
            Width of the time window for counting timestamp differences.
            The default is 50e3 (50 ns).
        cycle_length : float, optional
            Length of each acquisition cycle. The default is 4e9 (4 ms).

        Returns
        -------
        deltas_all : dict
            Dictionary containing timestamp differences for each pair of pixels.

        """

        # Dictionary for the timestamp differences, where keys are the
        # pixel numbers of the requested pairs

        deltas_all = []

        data_pix_1 = data[0]
        data_pix_2 = data[1]

        indices1 = data_pix_1[0]
        indices2 = data_pix_2[0]
        timestamps1 = data_pix_1[1]
        timestamps2 = data_pix_2[1]

        timestamps_1 = []
        timestamps_2 = []

        # Go over cycles, shifting the timestamps from each next
        # cycle by lengths of cycles before (e.g., for the 4th cycle
        # add 12 ms)
        for i, _ in enumerate(self.cycle_ends[:-1]):
            slice_from = self.cycle_ends[i]
            slice_to = self.cycle_ends[i + 1]
            pix1_slice = indices1[
                (indices1 >= slice_from) & (indices1 < slice_to)
            ]
            if not np.any(pix1_slice):
                continue
            pix2_slice = indices2[
                (indices2 >= slice_from) & (indices2 < slice_to)
            ]
            if not np.any(pix2_slice):
                continue

            # Shift timestamps by cycle length
            tmsp1 = timestamps1[np.isin(indices1, pix1_slice)]
            tmsp1 = tmsp1[tmsp1 > 0]
            tmsp1 = tmsp1 + cycle_length * i

            tmsp2 = timestamps2[np.isin(indices2, pix2_slice)]
            tmsp2 = tmsp2[tmsp2 > 0]
            tmsp2 = tmsp2 + cycle_length * i

            timestamps_1.extend(tmsp1)
            timestamps_2.extend(tmsp2)

        timestamps_1 = np.array(timestamps_1)
        timestamps_2 = np.array(timestamps_2)

        # Indicators for each pixel: 0 for timestamps from one pixel
        # 1 - from the other
        pix1_ind = np.zeros(len(timestamps_1), dtype=np.int32)
        pix2_ind = np.ones(len(timestamps_2), dtype=np.int32)

        pix1_data = np.vstack((pix1_ind, timestamps_1))
        pix2_data = np.vstack((pix2_ind, timestamps_2))

        # Dataframe for each pixel with pixel indicator and
        # timestamps
        df1 = pd.DataFrame(pix1_data.T, columns=["Pixel_index", "Timestamp"])
        df2 = pd.DataFrame(pix2_data.T, columns=["Pixel_index", "Timestamp"])

        # Combine the two dataframes
        df_combined = pd.concat((df1, df2), ignore_index=True)

        # Sort the timestamps
        df_combined.sort_values("Timestamp", inplace=True)

        # Subtract pixel indicators of neighbors; values of 0
        # correspond to timestamp differences for the same pixel
        # '-1' and '1' - to differences from different pixels
        df_combined["Pixel_index_diff"] = df_combined["Pixel_index"].diff()

        # Calculate timestamp difference between neighbors
        df_combined["Timestamp_diff"] = df_combined["Timestamp"].diff()

        # Get the correct timestamp difference sign
        df_combined["Timestamp_diff"] = (
            df_combined["Timestamp_diff"] * df_combined["Pixel_index_diff"]
        )

        # Collect timestamp differences where timestamps are from
        # different pixels
        filtered_df = df_combined[abs(df_combined["Pixel_index_diff"]) == 1]

        # Save only timestamps differences in the requested window
        delta_ts = filtered_df[
            abs(filtered_df["Timestamp_diff"]) < delta_window
        ]["Timestamp_diff"].values

        deltas_all.extend(delta_ts)

        return deltas_all

    def _calculate_timestamps_differences(self, args):
        """Calculate photon coincidences and save to '.feather'.

        Parameters
        ----------
        #TODO
        """

        print("From The Process", time.time())

        # try-except railguard for a function that goes to separate
        # cores
        file, data, pixel_pair = args
        try:
            # Check if the 'delta_ts_data' folder exists
            output_dir = Path(self.path) / "delta_ts_data"
            output_dir.mkdir(exist_ok=True)

            # Calculate the differences and convert them to a pandas
            # dataframe - standard approach sending the whole matrix of
            # 64xN to each child process
            # deltas_all = cd.calculate_differences_2212_fast(
            #     data, pixel_pair, self.pix_coor
            # )

            # Calculate the differences and convert them to a pandas
            # dataframe - pre-chunking the data down to 2 arrays
            deltas_all = self._calculate_differences_2212_fast(data)
            
            data_for_plot_df = pd.DataFrame(
                deltas_all, columns=[f"{pixel_pair[0]},{pixel_pair[1]}"]
            ).T

            # Save the data to a '.feather' file
            file_name = Path(
                file
            ).stem  # Get the file name without the extension
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            file_name = Path(file).stem
            output_file = (
                output_dir
                / f"{file_name}_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )
            ft.write_feather(
                data_for_plot_df.reset_index(drop=True), output_file
            )

        except Exception as e:
            print(f"Error processing file {file}: {e}")

    def _combine_feather_files(self, path_to_feather_files: str):

        os.chdir(path_to_feather_files)

        for pixel_pair in self.pixel_pairs:
            ft_files = glob.glob(f"*{pixel_pair[0]}_{pixel_pair[1]}*.feather")
            data_all = pd.DataFrame()
            for ft_file in ft_files:
                data = ft.read_feather(ft_file)
                data_all = pd.concat((data_all, data), ignore_index=True)
            data_all.to_feather(
                f"combined_{pixel_pair[0]}_{pixel_pair[1]}.feather"
            )

    # ## Heavy data chunking - fast calculation in each child process at
    # ## the cost of long chunking - no acceleration as a result
    # def _chunk_that_data_a_lot(self, data, pixel_pair, cycle_length: float = 4e9):

    #     pix_coor = np.arange(256).reshape(64, 4)

    #     # Find ends of cycles
    #     cycle_ends = np.argwhere(data[0].T[0] == -2)
    #     cycle_ends = np.insert(cycle_ends, 0, 0)

    #     tdc1, pix_c1 = np.argwhere(pix_coor == pixel_pair[0])[0]
    #     pix1 = np.where(data[tdc1].T[0] == pix_c1)[0]

    #     timestamps_1 = []
    #     timestamps_2 = []

    #     # Second pixel in the pair
    #     tdc2, pix_c2 = np.argwhere(pix_coor == pixel_pair[1])[0]
    #     pix2 = np.where(data[tdc2].T[0] == pix_c2)[0]

    #     # Go over cycles, shifting the timestamps from each next
    #     # cycle by lengths of cycles before (e.g., for the 4th cycle
    #     # add 12 ms)
    #     for i, _ in enumerate(cycle_ends[:-1]):
    #         slice_from = cycle_ends[i]
    #         slice_to = cycle_ends[i + 1]
    #         pix1_slice = pix1[(pix1 >= slice_from) & (pix1 < slice_to)]
    #         if not np.any(pix1_slice):
    #             continue
    #         pix2_slice = pix2[(pix2 >= slice_from) & (pix2 < slice_to)]
    #         if not np.any(pix2_slice):
    #             continue

    #         # Shift timestamps by cycle length
    #         tmsp1 = data[tdc1].T[1][pix1_slice]
    #         tmsp1 = tmsp1[tmsp1 > 0]
    #         tmsp1 = tmsp1 + cycle_length * i

    #         tmsp2 = data[tdc2].T[1][pix2_slice]
    #         tmsp2 = tmsp2[tmsp2 > 0]
    #         tmsp2 = tmsp2 + cycle_length * i

    #         timestamps_1.extend(tmsp1)
    #         timestamps_2.extend(tmsp2)

    #     timestamps_1 = np.array(timestamps_1)
    #     timestamps_2 = np.array(timestamps_2)

    #     return (timestamps_1, timestamps_2)

    def _chunk_that_data(self, data, pixel_pair, cycle_length: float = 4e9):
        """Chunk data down to 2 rows - prepare for child processes.

        Parameters
        ----------
        data : np.ndarray
            The whole matrix.
        pixel_pair : list
            Pair of pixels to slice out of the whole matrix.
        cycle_length : float, optional
            Cycle length in ps, by default 4e9

        Returns
        -------
        tuple
            Tuple of two arrays, each contains timestamps from the two
            pixels requested and the indices of the timestamps in the
            original matrix of data. The indices are used for correct
            assigning to the corresponding acquisition cycles.
        """

        tdc1, pix_c1 = np.argwhere(self.pix_coor == pixel_pair[0])[0]
        indices1 = np.where(data[tdc1].T[0] == pix_c1)[0]

        # Second pixel in the pair
        tdc2, pix_c2 = np.argwhere(self.pix_coor == pixel_pair[1])[0]
        indices2 = np.where(data[tdc2].T[0] == pix_c2)[0]

        data_cut_1 = np.array((indices1, data[tdc1].T[1][indices1]))
        data_cut_2 = np.array((indices2, data[tdc2].T[1][indices2]))

        return (data_cut_1, data_cut_2)

    def calculate_and_save_timestamp_differences_mp(self):

        # Find all LinoSPAD2 data files
        files = glob.glob("*.dat")

        if not files:
            raise ValueError("No .dat files found in the specified path.")

        self.pixel_pairs = []
        for i in self.pixels[0]:
            for j in self.pixels[1]:
                self.pixel_pairs.append([i, j])

        start_time = time.time()

        # Go file by file
        for file in files:
            print(f"Processing file: {file}")

            start_time_unpack = time.time()

            # Unpack the data from the file
            data = self._unpack_binary_data(file)

            # Pre-collect the indices of the acquisition cycles' ends
            self.cycle_ends = np.argwhere(data[0].T[0] == -2)
            self.cycle_ends = np.insert(self.cycle_ends, 0, 0)

            start_time_args = time.time()

            # Prepare the arguments for the child processes:
            # 'file' and 'pixel_pair' for naming purposes,
            # chunked data as a tuple of two arrays to work with
            args = [
                (
                    file,
                    self._chunk_that_data(data, pixel_pair),
                    pixel_pair,
                )
                for pixel_pair in self.pixel_pairs
            ]

            # print(f"Created args in {time.time() - start_time_args}")
            print("Before the processes", time.time())

            with multiprocessing.Pool(
                min(self.number_of_cores, len(self.pixel_pairs))
            ) as pool:
                pool.map(
                    self._calculate_timestamps_differences,
                    args,
                )

        end_time = time.time()

        print(
            f"Parallel processing of files "
            "files (with each writing to its file) finished "
            f"in: {round(end_time - start_time, 2)} s"
        )

        # Combine '.feather' files from separate cores
        path_to_feathers = os.path.join(self.path, "delta_ts_data")

        self._combine_feather_files(path_to_feathers)

        path_output = os.path.join(self.path, "delta_ts_data")

        print(
            "The feather files with the timestamp differences were "
            f"combined into the 'combined.feather' file in {path_output}"
        )


if __name__ == "__main__":
    # path = r"D:\LinoSPAD2\Data\B7d\2024.11.11\500t\MP_test"
    path = r'/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70'

    mp = MpWizard(
        path,
        # pixels=[[144], [171, 172]],
        pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
        # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
        daughterboard_number="B7d",
        motherboard_number="#28",
        firmware_version="2212b",
        timestamps=500,
        number_of_cores=5,
    )

    mp.calculate_and_save_timestamp_differences_mp()


### Standard approach - for control and comparison

import time

from daplis.functions import delta_t

time_start = time.time()

path = r"/media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70"


delta_t.calculate_and_save_timestamp_differences_fast(
    path,
    rewrite=True,
    # pixels=[144, 171],
    pixels=[[x for x in range(55, 60)], [x for x in range(175, 180)]],
    # pixels=[[x for x in range(20, 80)], [x for x in range(130, 190)]],
    daughterboard_number="B7d",
    motherboard_number="#28",
    firmware_version="2212b",
    timestamps=500,
)

print(f"Finished in {time.time() - time_start}")

Processing file: 0000054222.dat
Before the processes 1737459844.5941215
From The Process 1737459844.6574094
From The Process 1737459844.6683671
From The Process 1737459844.6839461
From The Process 1737459844.715067
From The Process 1737459844.7342224
From The Process 1737459845.1763504
From The Process 1737459845.1893883
From The Process 1737459845.2241685
From The Process 1737459845.374938
From The Process 1737459845.7645507
From The Process 1737459845.8894908
From The Process 1737459845.9427645
From The Process 1737459846.1090639
From The Process 1737459846.2308009
From The Process 1737459846.3279898
From The Process 1737459846.504545
From The Process 1737459846.5848405
From The Process 1737459846.6928904
From The Process 1737459846.989765
From The Process 1737459847.2066672
From The Process 1737459847.2768579
From The Process 1737459847.2806358
From The Process 1737459847.371996
From The Process 1737459847.5272338
From The Process 1737459847.797635
Parallel processing of files files


> > > Timestamp differences are saved as0000054222-0000054222.feather in /media/sj/King4TB/LS2_Data/2024.11.11/500t/MP_test_70/delta_ts_data < < <
Finished in 6.534499883651733
